# 구글 드라이브 마운트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 라이브러리 임포트

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np

# 데이터셋 로드 및 전처리

## 데이터셋 로드

In [3]:
df = pd.read_csv('/content/drive/MyDrive/MLP Lab/조교자료/codes/awe_1/AWE_1000_score.csv')
df.head()

,Unnamed: 0,id,nationality,sex,title,content,score
0,1,A100000_v01.txt_2.txt,중국,남자,우리 나라의 계절과 날씨,제 나라의 계절은 아주 좋아요. 네 계절이 있습니다. 봄에는 날씨가 좋아요. 꽃이 ...,80
1,2,A100000_v02.txt_1.txt,중국,여자,주말 이야기,어제 명동에 갔습니다. 친구하고 같이 갔습니다. 가게가 많습니다. 옷가게가 있었습니...,70
2,3,A100000_v03.txt_1.txt,중국,남자,여행 계획,저는 여행을 제주도에서 갈 싶습니다. 저는 친구 같이 갈 겁니다. 6월 15일 갈 ...,50
3,4,A100001_v01.txt_1.txt,중국,남자,주말 이야기,주말에 도서관에서 공부했어요. 친구하고 노래방에서 노래했어요. 친구하고 식당에서 비...,60
4,5,A100001_v02.txt_1.txt,중국,남자,주말 이야기,어제 저는 학교에 갔습니다. 학교에서 친구하고를 다닙니다. 학교에 학생 식당 있습니다.,20


In [4]:
corpus = df['content'].values
score = df['score'].values

## Character Dict 생성

In [5]:
all_context = ' '.join(corpus)
char_set = list(set(all_context))
char_dict = {char:idx for idx,char in enumerate(char_set,1)}
char_dict['<pad>']=0
char_dict

{'릅': 1,
 '판': 2,
 '버': 3,
 '헌': 4,
 '푼': 5,
 '수': 6,
 '렌': 7,
 '찜': 8,
 '찼': 9,
 '캠': 10,
 '끔': 11,
 '보': 12,
 '축': 13,
 '웁': 14,
 'a': 15,
 '찍': 16,
 '엄': 17,
 '냉': 18,
 '핸': 19,
 '펀': 20,
 '옛': 21,
 '녀': 22,
 '석': 23,
 '첮': 24,
 '침': 25,
 '읹': 26,
 'ㅇ': 27,
 '덜': 28,
 '짜': 29,
 '닥': 30,
 '높': 31,
 '챙': 32,
 'Q': 33,
 '캍': 34,
 '점': 35,
 '뮤': 36,
 '잉': 37,
 '훠': 38,
 '됨': 39,
 '퉁': 40,
 '겁': 41,
 '갑': 42,
 '란': 43,
 '훗': 44,
 '굥': 45,
 '파': 46,
 '얀': 47,
 '택': 48,
 '호': 49,
 '쌌': 50,
 '날': 51,
 '클': 52,
 '맬': 53,
 '혼': 54,
 '함': 55,
 '배': 56,
 '켜': 57,
 '안': 58,
 '로': 59,
 '료': 60,
 '(': 61,
 '혔': 62,
 '해': 63,
 '갈': 64,
 '색': 65,
 '싫': 66,
 '박': 67,
 '쁬': 68,
 '밝': 69,
 '샹': 70,
 '밖': 71,
 '>': 72,
 '짧': 73,
 'o': 74,
 '냐': 75,
 '땠': 76,
 '줄': 77,
 '암': 78,
 '맞': 79,
 '렵': 80,
 '씬': 81,
 '팠': 82,
 '괜': 83,
 '롯': 84,
 'r': 85,
 '플': 86,
 '응': 87,
 '널': 88,
 '람': 89,
 '팁': 90,
 '듭': 91,
 '탙': 92,
 'n': 93,
 '밮': 94,
 '었': 95,
 '꿨': 96,
 '곳': 97,
 '입': 98,
 '먹': 99,
 '목': 100,
 '깄': 10

# 모델 구현

In [48]:
class Net(nn.Module):
  def __init__(self,vocab_size,input_dim,hidden_dim,num_layers):
    super(Net,self).__init__()

    self.vocab_size = vocab_size
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.num_layers = num_layers

    self.embed = nn.Embedding(vocab_size,input_dim,padding_idx=0)
    self.rnn = nn.RNN(input_dim,hidden_dim,num_layers,batch_first=True)
    self.fc = nn.Linear(hidden_dim,1)


  def forward(self,x):
    output = self.embed(x)
    output,hidden = self.rnn(output)
    output = self.fc(hidden[-1])
    return output

## 모델 추론 테스트

In [27]:
sample = [char_dict[c] for c in corpus[0]]
one_hot_sample = torch.LongTensor(sample).unsqueeze(0)
print(one_hot_sample.shape)
output = model(one_hot_sample)
print(output)

torch.Size([1, 267])
tensor([[0.0662]], grad_fn=<AddmmBackward0>)


# 데이터셋 객체 만들기

In [28]:
max_length = max([len(c) for c in corpus])
max_length

574

In [29]:
class CustomDataset(Dataset):
  def __init__(self,X,Y):
    self.input = X
    self.label = Y

  def __len__(self):
    return len(self.label)

  def __getitem__(self,idx):
    input = torch.LongTensor(self.input[idx]).to('cuda:0')
    label = torch.FloatTensor([self.label[idx]]).to('cuda:0')

    return input,label

In [30]:
X_char_data = []

for c in corpus:
  temp = (max_length - len(c))*[char_dict['<pad>']]+[char_dict[t] for t in c]
  X_char_data.append(temp)

y_data = score
train_dataset = CustomDataset(X_char_data,y_data)

In [31]:
train_dataset[0]

(tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

In [98]:
train_dataloader = DataLoader(train_dataset,batch_size=10)

# Train

In [108]:
model = Net(len(char_dict),512,1024,4)
model.to('cuda:0')
optimizer = optim.Adam(model.parameters(),lr=0.0001)

In [109]:
 for x_batch,y_batch in train_dataloader:
  pred=model(x_batch)
  print(pred)

tensor([[ 0.0862],
        [ 0.0193],
        [-0.0461],
        [ 0.0091],
        [-0.0109],
        [-0.0343],
        [ 0.0223],
        [-0.0324],
        [ 0.0011],
        [-0.0385]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0337],
        [-0.0151],
        [ 0.0012],
        [-0.0362],
        [-0.0604],
        [-0.0426],
        [-0.0375],
        [-0.0102],
        [-0.0102],
        [-0.0366]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[ 0.0022],
        [-0.0238],
        [-0.0286],
        [-0.0258],
        [ 0.0012],
        [-0.0170],
        [-0.0112],
        [-0.0347],
        [-0.0698],
        [ 0.0021]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-0.0733],
        [-0.0991],
        [-0.0504],
        [-0.0109],
        [-0.0308],
        [-0.0024],
        [-0.0074],
        [-0.0259],
        [-0.0098],
        [ 0.0772]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor([[-4.4155e-02],
        [-7.5348e-02],
        [-9.3779e-

KeyboardInterrupt: 

In [110]:
num_epochs = 10

for i in range(num_epochs):
  loss_li=[]
  optimizer.zero_grad()
  for x_batch,y_batch in train_dataloader:
    pred_y = model(x_batch)
    loss = nn.functional.mse_loss(pred_y,y_batch)
    loss.backward()
    loss_li.append(loss.cpu().detach().numpy())
    optimizer.step()

  print(f'num_epochs:{i}, loss : {np.mean(loss_li)}')

num_epochs:0, loss : 3148.83251953125
num_epochs:1, loss : 1954.5548095703125
num_epochs:2, loss : 1347.3114013671875
num_epochs:3, loss : 957.35302734375
num_epochs:4, loss : 706.58251953125
num_epochs:5, loss : 551.7200927734375
num_epochs:6, loss : 461.5219421386719
num_epochs:7, loss : 412.0495300292969
num_epochs:8, loss : 386.9548034667969
num_epochs:9, loss : 375.81622314453125


In [ ]:
model(X_train[0])